In [1]:
import sys
import os
import cudf  #CUDA计算
import pandas as pd

# 自行编写的包
sys.path.append('/home/ubuntu/notebook/Investor-Sentiment')
from loader.findata_loader import Loader
from utils.sql import DB

# ------------------------------数据集路径----------------------------------#
DATASETS_PATH = '/data/DataSets/investor_sentiment/'

提取自然日期

In [2]:
def extract_pdate_wcsentiment():
        """
        提取推文中的情绪
        :return:
        """
        # 提取
        df_select = pd.read_sql(
                "SELECT id,p_date, title_neg, cover_neg, biz FROM WECAHT_DATA.articles_tag "
                "WHERE mov=10 AND biz in ('MjM5MzMwNjM0MA==','MjM5NzQ5MTkyMA==','MjY2NzgwMjU0MA==','MjY2NzgwMjU0MA==')",
                con=DB().ENGINE,
                parse_dates=["p_date"])

        # 筛选
        df_select['trade_date'] = df_select['p_date'].dt.strftime('%Y%m%d').astype('uint32')
        df_select = df_select.query("trade_date>=20140101")

        # 阈值处理
        df_select['title_flag'] = df_select['title_neg'].apply(lambda x: 1 if x > 0.5 else 0)
        df_select['cover_flag'] = df_select['cover_neg'].apply(lambda x: 1 if x > 0.5 else 0)

        # 每日情绪
        df_select['day_article'] = df_select.groupby('trade_date')['id'].transform('count')

        df_select['tex_neg'] = df_select['title_flag'] / df_select['day_article']
        df_select['img_neg'] = df_select['cover_flag'] / df_select['day_article']

        # 筛选
        return df_select[['trade_date', 'img_neg', 'tex_neg']].set_index('trade_date').sort_index().reset_index()


extract_pdate_wcsentiment()

/tmp/ipykernel_26677/2172678335.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_select['title_flag'] = df_select['title_neg'].apply(lambda x: 1 if x > 0.5 else 0)
/tmp/ipykernel_26677/2172678335.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_select['cover_flag'] = df_select['cover_neg'].apply(lambda x: 1 if x > 0.5 else 0)
/tmp/ipykernel_26677/2172678335.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

,trade_date,img_neg,tex_neg
0,20140101,0.0,0.0
1,20140102,0.0,0.0
2,20140102,0.0,0.0
3,20140102,0.0,0.0
4,20140103,0.0,0.0
...,...,...,...
26633,20220626,0.0,0.0
26634,20220626,0.0,0.0
26635,20220626,0.0,0.0
26636,20220626,0.0,0.0
